In [4]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-01.parquet
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet

In [76]:
# !pip install pyarrow
# !pip install sklearn

In [99]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error

In [128]:
jan = pd.read_parquet('fhv_tripdata_2021-01.parquet')

Question 1:

In [7]:
jan.shape[0]

1154112

In [8]:
jan.columns

Index(['dispatching_base_num', 'pickup_datetime', 'dropOff_datetime',
       'PUlocationID', 'DOlocationID', 'SR_Flag', 'Affiliated_base_number'],
      dtype='object')

In [129]:
jan['duration'] = jan.dropOff_datetime - jan.pickup_datetime

In [130]:
jan.duration = jan.duration.apply(lambda td: td.total_seconds()/60)

Question 2:

In [40]:
jan.duration.mean()

19.167224093791006

In [131]:
jan_limited = jan[(jan.duration >= 1) & (jan.duration <= 60)]

Question 2.5:

In [132]:
jan.shape[0]-jan_limited.shape[0]

44286

In [59]:
jan_limited.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1109826 entries, 0 to 1154111
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   dispatching_base_num    1109826 non-null  object        
 1   pickup_datetime         1109826 non-null  datetime64[ns]
 2   dropOff_datetime        1109826 non-null  datetime64[ns]
 3   PUlocationID            1109826 non-null  float64       
 4   DOlocationID            1109826 non-null  float64       
 5   SR_Flag                 0 non-null        object        
 6   Affiliated_base_number  1109053 non-null  object        
 7   duration                1109826 non-null  float64       
dtypes: datetime64[ns](2), float64(3), object(3)
memory usage: 76.2+ MB


In [133]:
# jan_limited.fillna({"DOlocationID":-1,'PUlocation':-1}, inplace=True)
jan_limited['PUlocationID']=jan_limited['PUlocationID'].fillna(-1)
jan_limited['DOlocationID']=jan_limited['DOlocationID'].fillna(-1)

<ipython-input-133-cecfd7feaa17>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['PUlocationID']=jan_limited['PUlocationID'].fillna(-1)
<ipython-input-133-cecfd7feaa17>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['DOlocationID']=jan_limited['DOlocationID'].fillna(-1)


In [134]:
len(jan_limited.loc[jan_limited['DOlocationID'] == -1])

147907

Question 3:

In [135]:
len(jan_limited.loc[jan_limited['PUlocationID'] == -1])/len(jan_limited['PUlocationID'])

0.8352732770722617

In [126]:
jan_limited.shape

(1109826, 8)

In [84]:
categorical = ['PUlocationID','DOlocationID']

In [82]:
jan_limited.dtypes

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                      object
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [83]:
jan_limited['PUlocationID']=jan_limited['PUlocationID'].astype(str)
jan_limited['DOlocationID']=jan_limited['DOlocationID'].astype(str)

<ipython-input-83-ea0f7957599f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['PUlocationID']=jan_limited['PUlocationID'].astype(str)
<ipython-input-83-ea0f7957599f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jan_limited['DOlocationID']=jan_limited['DOlocationID'].astype(str)


In [79]:
dv = DictVectorizer()

dispatching_base_num              object
pickup_datetime           datetime64[ns]
dropOff_datetime          datetime64[ns]
PUlocationID                     float64
DOlocationID                     float64
SR_Flag                           object
Affiliated_base_number            object
duration                         float64
dtype: object

In [156]:
train_dict = jan_limited[categorical].to_dict(orient='records')

In [149]:
dv.fit(train_dict)

DictVectorizer()

In [152]:
len(dv.get_feature_names())

526

In [157]:
X_train = dv.fit_transform(train_dict)

Question 4:

In [90]:
X_train.shape[1]

525

In [158]:
target = 'duration'
y_train = jan_limited[target].values

In [159]:
from sklearn.linear_model import LinearRegression

In [160]:
lr = LinearRegression()
lr.fit(X_train,y_train)

LinearRegression()

In [161]:
y_pred = lr.predict(X_train)

Question 5:

In [162]:
mean_squared_error(y_train,y_pred,squared=False)

11.415432830521663

In [137]:
feb = pd.read_parquet('fhv_tripdata_2021-02.parquet')

In [138]:
feb['duration'] = feb.dropOff_datetime - feb.pickup_datetime

In [139]:
feb.duration = feb.duration.apply(lambda td: td.total_seconds()/60)

In [140]:
# jan_limited.fillna({"DOlocationID":-1,'PUlocation':-1}, inplace=True)
feb_limited['PUlocationID']=feb_limited['PUlocationID'].fillna(-1)
feb_limited['DOlocationID']=feb_limited['DOlocationID'].fillna(-1)

In [141]:
feb_limited = feb[(feb.duration >= 1) & (feb.duration <= 60)]

In [142]:
feb_limited['PUlocationID']=feb_limited['PUlocationID'].astype(str)
feb_limited['DOlocationID']=feb_limited['DOlocationID'].astype(str)

<ipython-input-142-726b2329b9cb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['PUlocationID']=feb_limited['PUlocationID'].astype(str)
<ipython-input-142-726b2329b9cb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feb_limited['DOlocationID']=feb_limited['DOlocationID'].astype(str)


In [143]:
jan_train_dict = jan_limited[categorical].to_dict(orient='records')
feb_train_dict = feb_limited[categorical].to_dict(orient='records')

In [144]:
X_val = dv.fit_transform(feb_train_dict)
y_val = feb_limited[target].values

In [145]:
y_val_pred = lr.predict(X_val)

ValueError: X has 526 features, but LinearRegression is expecting 525 features as input.

In [166]:
def prep_file(file):
    df = pd.read_parquet(file)
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)
    df['PUlocationID']=df['PUlocationID'].fillna(-1)
    df['DOlocationID']=df['DOlocationID'].fillna(-1)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df
    

In [165]:
def make_predictions(df):
    categorical = ['PUlocationID','DOlocationID']
    target = 'duration'
    x_dict = df[categorical].to_dict(orient='records')
    X_Train = dv.fit_transform(x_dict)
    y_train = df[target].values
    lr = LinearRegression()
    lr.fit(X_train,y_train)
    return lr.predict(X_train)

In [167]:
y_preds = make_predictions(prep_file('fhv_tripdata_2021-01.parquet'))

In [ ]:
y_train = prep_file('fhv_tripdata_2021-01.parquet').duration.values

In [168]:
mean_squared_error(y_train,y_preds,squared=False)

11.415432830521663